## Installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# !cd drive
# !cd MyDrive/IP_S7/Video

Mounted at /content/drive


In [ ]:
cd drive

/content/drive


In [ ]:
cd "MyDrive/Data Science Project/Data"

/content/drive/MyDrive/Data Science Project/Data


## Imports

In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.3 MB/s eta 0:00:00


In [ ]:
from io import open
import string
import re
import random
import json
import torch
import copy
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import pickle
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
import os


## DataLoader

In [ ]:
model_path = "siebert/sentiment-roberta-large-english"

In [ ]:
class CityDataset(Dataset):
  def __init__(self, tokenizer, data):

    super(CityDataset, self).__init__()
    self.data = data
    self.tokenizer=tokenizer

  def __len__(self):

    return len(self.data)

  def __getitem__(self, index):

    text = self.data[index][0]
    inputs = self.tokenizer(text ,truncation=True,padding='max_length', return_tensors='pt')
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    return {
      'ids': ids[0],
      'mask': mask[0],
      'index': index
    }

BATCH_SIZE = 128
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
sm = torch.nn.Softmax(dim=1)

In [ ]:
dir = "Cities_Processed/"
num = 0
# Added only two cities for this script as we were running multiple instances with different cities to save time
ct = ["london.csv", "san-francisco.csv"]
for city in ct:
  # text = ""
  print(f"City: {city}")
  fname = os.path.join(dir, city)
  if os.path.isfile(fname):
    df = pd.read_csv(fname)
    data = df.values.tolist()
    unlb_data = CityDataset(tokenizer, data=data)
    dataloader = DataLoader(dataset= unlb_data, batch_size=BATCH_SIZE, shuffle=False)
    scores = []
    sentiment = []
    index = []
    for inputs in tqdm(dataloader):
      with torch.no_grad():
        out = model(input_ids = inputs['ids'].to(device), attention_mask = inputs['mask'].to(device))
        preds = out.logits.argmax(-1).cpu()
        sentiment.extend(pd.Series(preds).map(model.config.id2label).values.tolist())
        scores.extend(sm(out.logits)[:,1].tolist())
        index.extend(inputs['index'].tolist())
    data = [[a[0],b,c] for (a,b,c) in zip(data, sentiment, scores)]
    for i in range(len(index)):
      if i != index[i]:
        print("Data not sequential")
        break
    print(f"Length = {len(index)}")
    df = pd.DataFrame(data, columns=("Review","Sentiment", "Score"))
    df.to_csv(f"Cities_Labelled/{city}", index=False)

City: london.csv


  0%|          | 0/617 [00:00<?, ?it/s]

Length = 78946
City: san-francisco.csv


  0%|          | 0/237 [00:00<?, ?it/s]

Length = 30282
